<a href="https://colab.research.google.com/github/Analgore17/weather_apps/blob/main/foresta_billing_pwa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 🚜 Dhaba Pro: WhatsApp Direct + Receipt Style PDF
import os
import subprocess
import time
import sys

# --- 1. INSTALL DEPENDENCIES ---
print("⏳ Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "streamlit", "fpdf", "pandas"], stdout=subprocess.DEVNULL)
print("✅ Dependencies installed.")

# --- 2. CREATE THE APP FILE (app.py) ---
print("📝 Creating application file...")

app_code = """
import streamlit as st
import pandas as pd
from fpdf import FPDF
from datetime import datetime
import urllib.parse

# --- PAGE CONFIG ---
st.set_page_config(page_title="Dhaba Bill Manager", page_icon="🥘", layout="wide")

# --- CUSTOM CSS ---
st.markdown(\"\"\"
    <style>
    .stButton>button { background-color: #25D366; color: white; border-radius: 8px; width: 100%; font-weight: bold;}
    .stTextInput>div>div>input { border-radius: 5px; }
    </style>
\"\"\", unsafe_allow_html=True)

# --- SESSION STATE ---
if 'logged_in' not in st.session_state: st.session_state['logged_in'] = False
if 'menu' not in st.session_state: st.session_state['menu'] = {}
if 'cart' not in st.session_state: st.session_state['cart'] = []

# --- FUNCTIONS ---

def generate_receipt_pdf(cart, total, name, mobile):
    # Custom 'Thermal Receipt' style size (approx 80mm width)
    pdf = FPDF(format='A5')
    pdf.add_page()

    # Title
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, "SHER-E-PUNJAB DHABA", ln=1, align='C')

    pdf.set_font("Arial", size=10)
    pdf.cell(0, 5, "Delicious Desi Food", ln=1, align='C')
    pdf.line(10, 25, 138, 25)

    # Customer Details
    pdf.ln(10)
    pdf.set_font("Courier", 'B', 10)
    pdf.cell(0, 5, f"Date: {datetime.now().strftime('%d-%m-%Y %H:%M')}", ln=1)
    pdf.cell(0, 5, f"Customer: {name}", ln=1)
    pdf.cell(0, 5, f"Mobile: {mobile}", ln=1)
    pdf.ln(5)

    # Table Header
    pdf.set_font("Arial", 'B', 10)
    pdf.cell(60, 8, "Item", 1)
    pdf.cell(20, 8, "Qty", 1, align='C')
    pdf.cell(20, 8, "Price", 1, align='C')
    pdf.cell(30, 8, "Total", 1, align='C')
    pdf.ln()

    # Items
    pdf.set_font("Arial", size=10)
    for item in cart:
        pdf.cell(60, 8, item['Item'], 1)
        pdf.cell(20, 8, str(item['Qty']), 1, align='C')
        pdf.cell(20, 8, str(item['Price']), 1, align='C')
        pdf.cell(30, 8, str(item['Total']), 1, align='C')
        pdf.ln()

    # Total
    pdf.ln(5)
    pdf.set_font("Arial", 'B', 14)
    pdf.cell(80, 10, "Grand Total:", 0, 0, 'R')
    pdf.cell(50, 10, f"Rs. {total}", 0, 1, 'R')

    # Footer
    pdf.ln(10)
    pdf.set_font("Arial", 'I', 8)
    pdf.cell(0, 5, "Thank you! Visit Again.", ln=1, align='C')

    return pdf.output(dest='S').encode('latin-1')

def login():
    st.markdown("<h2 style='text-align:center;'>🔐 System Login</h2>", unsafe_allow_html=True)
    c1, c2, c3 = st.columns([1,2,1])
    with c2:
        u = st.text_input("User ID")
        p = st.text_input("Password", type="password")
        if st.button("Login"):
            if u == "admin" and p == "dhaba123":
                st.session_state['logged_in'] = True
                st.rerun()
            else: st.error("Invalid")

def main():
    # Sidebar
    st.sidebar.title("🥘 Menu")
    nav = st.sidebar.radio("Navigation", ["New Bill", "Manage Menu", "Logout"])

    if nav == "Logout":
        st.session_state['logged_in'] = False
        st.rerun()

    # --- MANAGE MENU ---
    elif nav == "Manage Menu":
        st.title("🛠️ Manage Products")

        c1, c2 = st.columns(2)
        with c1:
            st.subheader("Add Item")
            cats = list(st.session_state['menu'].keys())

            # Category Selection
            cat_mode = st.radio("Category", ["Select Existing", "New Category"], horizontal=True)
            if cat_mode == "Select Existing" and cats:
                category = st.selectbox("Choose Category", cats)
            else:
                category = st.text_input("Enter Category Name (e.g., Paneer Items)")

            name = st.text_input("Item Name")
            price = st.number_input("Price (Rs)", min_value=0, value=100)

            if st.button("💾 Save Item"):
                if category and name:
                    if category not in st.session_state['menu']: st.session_state['menu'][category] = {}
                    st.session_state['menu'][category][name] = price
                    st.success(f"Saved: {name}")
                    time.sleep(1)
                    st.rerun()

        with c2:
            st.subheader("Current Menu")
            st.json(st.session_state['menu'])

    # --- NEW BILLING ---
    elif nav == "New Bill":
        st.title("🧾 New Bill Invoice")

        if not st.session_state['menu']:
            st.error("Menu is empty! Please add items in 'Manage Menu' first.")
            st.stop()

        # 1. Customer Details
        col_cust1, col_cust2 = st.columns(2)
        with col_cust1:
            cust_name = st.text_input("Customer Name", placeholder="e.g. Rahul Kumar")
        with col_cust2:
            cust_mobile = st.text_input("Customer Mobile", placeholder="e.g. 9876543210")

        st.divider()

        # 2. Add Items
        c1, c2, c3, c4 = st.columns([2, 2, 1, 1])
        with c1:
            cat = st.selectbox("Category", list(st.session_state['menu'].keys()))
        with c2:
            item = st.selectbox("Item", list(st.session_state['menu'][cat].keys()))
            unit_price = st.session_state['menu'][cat][item]
        with c3:
            qty = st.number_input("Qty", 1, 50, 1)
        with c4:
            st.write(f"Price: {unit_price}")
            if st.button("➕ Add"):
                st.session_state['cart'].append({
                    "Item": item, "Price": unit_price, "Qty": qty, "Total": unit_price*qty
                })

        # 3. Bill Preview & Actions
        if st.session_state['cart']:
            st.divider()
            df = pd.DataFrame(st.session_state['cart'])
            st.dataframe(df, use_container_width=True)

            total = df['Total'].sum()
            st.markdown(f"<h3 style='text-align: right;'>Total Bill: Rs. {total}</h3>", unsafe_allow_html=True)

            # --- ACTIONS ROW ---
            act1, act2, act3 = st.columns(3)

            # A. PDF
            with act1:
                if cust_name and cust_mobile:
                    pdf_data = generate_receipt_pdf(st.session_state['cart'], total, cust_name, cust_mobile)
                    st.download_button("📥 Download PDF Receipt", data=pdf_data, file_name=f"Bill_{cust_name}.pdf", mime="application/pdf")
                else:
                    st.warning("Enter Name/Mobile for PDF")

            # B. WHATSAPP (SPECIFIC NUMBER)
            with act2:
                if cust_mobile and cust_name:
                    # Constructing the Exact Message Format Requested
                    msg_lines = []
                    msg_lines.append(f"*Dhaba Bill Notice 🥘*")
                    msg_lines.append(f"Dear {cust_name},")
                    msg_lines.append("Your order has been recorded.")
                    msg_lines.append("") # Empty line

                    for i in st.session_state['cart']:
                        msg_lines.append(f"- {i['Item']}: {i['Qty']} x {i['Price']} = {i['Total']}")

                    msg_lines.append("")
                    msg_lines.append(f"*Total Bill: Rs. {total}*")
                    msg_lines.append(f"Outstanding Balance: Rs. {total}")
                    msg_lines.append("")
                    msg_lines.append("Please pay at your earliest convenience. Thank you!")

                    final_msg = "%0A".join(msg_lines) # %0A is newline in URL

                    # Ensure mobile has country code (Defaulting to India 91 if not present)
                    target_number = cust_mobile.replace(" ", "").replace("+", "")
                    if len(target_number) == 10:
                        target_number = "91" + target_number

                    wa_link = f"https://wa.me/{target_number}?text={final_msg}"

                    st.markdown(f"<a href='{wa_link}' target='_blank'><button style='background-color:#25D366; color:white; border:none; padding:10px; border-radius:5px; width:100%; cursor:pointer;'>📲 Send on WhatsApp</button></a>", unsafe_allow_html=True)
                else:
                    st.warning("Enter Name/Mobile for WhatsApp")

            # C. RESET
            with act3:
                if st.button("🗑️ Reset Bill"):
                    st.session_state['cart'] = []
                    st.rerun()

if __name__ == '__main__':
    if st.session_state['logged_in']:
        main()
    else:
        login()
"""

with open("app.py", "w") as f:
    f.write(app_code)

print("✅ App file created.")

# --- 3. RUN SERVER ---
print("\n🚀 Starting Server...")
public_ip = subprocess.check_output(["curl", "ipv4.icanhazip.com"]).decode("utf-8").strip()
print(f"🔑 PASSWORD: \033[1m{public_ip}\033[0m")

subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])
time.sleep(3)

try:
    process = subprocess.Popen(["npx", "localtunnel", "--port", "8501"], stdout=subprocess.PIPE)
    for line in iter(process.stdout.readline, b''):
        print(f"🌍 CLICK HERE: {line.decode('utf-8').strip()}")
        break
except:
    print("❌ Error starting tunnel.")

while True:
    time.sleep(10)

⏳ Installing dependencies...
✅ Dependencies installed.
📝 Creating application file...
✅ App file created.

🚀 Starting Server...
🔑 PASSWORD: 35.188.235.11
🌍 CLICK HERE: your url is: https://smooth-hats-argue.loca.lt
